<a href="https://colab.research.google.com/github/oussamasmihrou/food_kaggle_data/blob/main/food_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# @title 0.1 Utilities
# We will include only the packages needed to run the code instead
# of the whole libraries for optimization purposes.
# We'll also use typing and PEP 8 for variable control and code readability.


from pygments.formatters import HtmlFormatter
from plotly.graph_objects import Figure, Bar
from IPython.display import display, HTML
from pygments.lexers import PythonLexer
from pandas import DataFrame, read_csv
from typing import List, Optional
from plotly.express import bar
from google.colab import drive
from pygments import highlight
from google.colab import files


# The custom display class is a tool we programmed for this project
# to style our display snippets and mix code output with text for a more readable report.

class CustomDisplay:
    def __init__(self, font_family: str = "'Fira Code', monospace", default_text_color: str = "#393E46") -> None:
        self.font_family = font_family
        self.default_text_color = default_text_color

        # Define base style (including body style with white background)
        self.base_style: str = f"""
        <link rel="preconnect" href="https://fonts.googleapis.com">
        <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
        <link href="https://fonts.googleapis.com/css2?family=Fira+Code&display=swap" rel="stylesheet">
        <style>
            body {{
                background-color: #ffffff !important;  /* White background */
                margin: 0;
                font-family: {self.font_family};
                color: {self.default_text_color};
            }}
            .section_titles {{
                text-align: center;
                font-family: {self.font_family};
                font-size: 36px;
                font-weight: 500;
                padding: 10px;
                border-radius: 3px;
                background-color: #393E46;  /* Light grey background */
                margin-bottom: 15px;
                color: #F5F7F8;
                margin-right: 3px;
                padding: 10px;
            }}
            .section_paragraphs {{
                font-family: {self.font_family};
                font-size: 13px;
                margin-bottom: 10px;
                color: #333333;  /* Default text color */
            }}
            .section_subtitles {{
                font-family: {self.font_family};
                font-size: 24px;
                font-weight: bold;
                margin-bottom: 5px;
                color: #333333;  /* Default text color */
            }}
            .section_code {{
                font-family: {self.font_family};
                background-color: #f5f5f5;  /* Light background for code */
                color: #333333;  /* Dark text color for better contrast */
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 15px;
                border: 1px solid #dddddd;  /* Light grey border */
            }}
            .codehilite {{
                background: #f5f5f5;  /* Light background for code highlights */
                color: #333333;  /* Dark text color */
                padding: 15px;
                border-radius: 5px;
                white-space: pre-wrap;
                word-wrap: break-word;
            }}
        </style>"""

    def title(self, text: str = "Hello world!") -> None:
        """Displays a main title."""
        html_content: str = f"<div class='section_titles'>{text}</div>"
        display(HTML(self.base_style + html_content))

    def subtitle(
        self,
        text: str,
        color: str = None,
        font_size: str = "24px",
        background_color: str = None,
        is_bold: bool = False,
        center: bool = False
    ) -> None:
        """Displays a secondary title with customizable text color, font size, background color, and alignment."""
        alignment = "center" if center else "left"
        html_content: str = (
            f"<div class='section_subtitles' style='background-color:White;"
            f"font-size:{font_size}; color:{color};background-color: {background_color};"
            f" padding: 2px; text-align:{alignment};'>{text}</div>"
        )
        display(HTML(self.base_style + html_content))

    def paragraph(
        self,
        text: str,
        color: str = None,
        font_size: str = "14px",
        background_color: str = None,
        is_bold: bool = False,
        font_weight: str = "300"
    ) -> None:
        """Displays a paragraph with customizable text color and alignment."""
        html_content: str = (
            f"<div class='section_paragraphs' style='background-color:White;"
            f" color:{color};background-color: {background_color};"
            f" font-weight: {font_weight}; padding: 5px;font-size:{font_size}'>{text}</div>"
        )
        display(HTML(self.base_style + html_content))

    def code(self, code: str = "", language: str = "python") -> None:
        """Displays formatted code using Pygments with a light theme and keyword highlighting."""
        lexer = PythonLexer()
        formatter = HtmlFormatter(
            style="default",  # Use the default light theme for code
            full=True,
            linenos=True,  # Show line numbers for better code structure
            cssclass="codehilite"
        )
        formatted_code: str = highlight(code, lexer, formatter)
        html_content: str = f"""
        <style>
            .overflow_scroll {{
                border-radius: 5px;
                overflow-x: auto;
                white-space: nowrap;
            }}
        </style>
        <div class='overflow_scroll section_code'>{formatted_code}</div>
        """
        display(HTML(self.base_style + html_content))


# Example usage of custom displayer
# cdisplay = CustomDisplay(font_family="'Fira Code', monospace", default_text_color="#393E46")
# cdisplay.title("Section 1 - Data Exploration")
# cdisplay.subtitle("This is a centered subtitle", color="Red", is_bold=True, font_size="40px", center=True)
# cdisplay.subtitle("This is a left-aligned subtitle", color="Blue", is_bold=False, font_size="24px", center=False)
# cdisplay.paragraph("""
# Data exploration is the first step in the data analysis process, where analysts or data scientists investigate a dataset to understand its structure, characteristics, and patterns. The goal is to gain insights into the data and prepare it for further analysis or modeling.
# """)
# cdisplay.code("""
# def binary_search(query, ids, start, stop):
#     if ids[start] == query:
#         return start
#     elif stop - start <= 1:
#         return -1
#     split = start + (stop - start) // 2
#     if ids[split] > query:
#         return binary_search(query, ids, start, split)
#     else:
#         return binary_search(query, ids, split, stop)
# """)

In [5]:
# @title 1.1 Initiating Data
#Initiation du Custum displayer
cdisplay = CustomDisplay(font_family = "'Fira Code', monospace", default_text_color = "#393E46" )
cdisplay.title("Section 1 - Exploration des données")

#Option 1 : Loading the csv file from google drive ( Recommended )

#Upload the file in your google drive root directory.
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/fr.openfoodfacts.org.products.csv'

# The low_memory=False parameter ensures that Pandas reads the entire CSV file at once
# rather than processing it in smaller chunks.

data = read_csv(file_path, delimiter='\t', low_memory = False)




#OPTION 2 : load from computer ( it might reaaally take a while)
'''
try:
  # The low_memory=False parameter ensures that Pandas reads the entire CSV file at once
# rather than processing it in smaller chunks.
  cdisplay.paragraph("Upload the fr.openfoodfacts.org.products.csv file")
  data = read_csv("fr.openfoodfacts.org.products.csv", delimiter='\t', low_memory = False)
except:
  data_csv = files.upload()  # uploading file
'''


cdisplay.paragraph(display(data.head()))




Mounted at /content/drive


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,0000000004559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,0000000016087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,0000000016094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# @title 1.2 Data Description
# DATA Descrition
cdisplay.subtitle("1.1 - Data description", is_bold=True, font_size="22px")
init_column = len(data.columns)
init_len = len(data)

# Display initial shape of the data

# Display summary statistics of the data

cdisplay.paragraph("Initial shape: " + str(data.shape), font_size="18px")
cdisplay.paragraph(" Data Frame Description: ", is_bold=True, font_size="18px", color = 'Red')
cdisplay.paragraph(display(data.describe()))


# Display the unique data types using the custom displayer
unique_data_types = data.dtypes.unique()

cdisplay.paragraph("Unique data types in the dataset: "+ str(unique_data_types), font_size="18px")

# Display the number of missing values per column
missing_values = data.isnull().sum()
cdisplay.paragraph("Missing values per column:", is_bold=True, font_size="20px")
cdisplay.paragraph(display(missing_values.head(15)), font_size="14px")


# Display the count of categorical values in each column (for categorical columns)
categorical_columns = data.select_dtypes(include=['object', 'category']).columns
if len(categorical_columns) > 0:
    categorical_count = data[categorical_columns].nunique()
    cdisplay.paragraph("Count of unique values in categorical columns:", is_bold=True, font_size="20px")
    cdisplay.paragraph(display(categorical_count.head(15)), font_size="14px")

,no_nutriments,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,nutrition_grade_uk,energy_100g,energy-from-fat_100g,fat_100g,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,0.0,248939.000000,248939.000000,0.0,248939.000000,0.0,0.0,2.611130e+05,857.000000,243891.000000,...,49.000000,3036.000000,165.000000,948.000000,0.0,268.000000,221210.000000,221210.000000,0.0,0.0
mean,NaN,1.936024,0.019659,NaN,0.055246,NaN,NaN,1.141915e+03,585.501214,12.730379,...,6.425698,31.458587,15.412121,49.547785,NaN,341.700764,9.165535,9.058049,NaN,NaN
std,NaN,2.502019,0.140524,NaN,0.269207,NaN,NaN,6.447154e+03,712.809943,17.578747,...,2.047841,31.967918,3.753028,18.757932,NaN,425.211439,9.055903,9.183589,NaN,NaN
min,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,8.000000,6.000000,NaN,0.000000,-15.000000,-15.000000,NaN,NaN
25%,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,3.770000e+02,49.400000,0.000000,...,6.300000,0.000000,12.000000,32.000000,NaN,98.750000,1.000000,1.000000,NaN,NaN
50%,NaN,1.000000,0.000000,NaN,0.000000,NaN,NaN,1.100000e+03,300.000000,5.000000,...,7.200000,23.000000,15.000000,50.000000,NaN,195.750000,10.000000,9.000000,NaN,NaN
75%,NaN,3.000000,0.000000,NaN,0.000000,NaN,NaN,1.674000e+03,898.000000,20.000000,...,7.400000,51.000000,15.000000,64.250000,NaN,383.200000,16.000000,16.000000,NaN,NaN
max,NaN,31.000000,2.000000,NaN,6.000000,NaN,NaN,3.251373e+06,3830.000000,714.290000,...,8.400000,100.000000,25.000000,100.000000,NaN,2842.000000,40.000000,40.000000,NaN,NaN


,0
code,23
url,23
creator,2
created_t,3
created_datetime,9
last_modified_t,0
last_modified_datetime,0
product_name,17762
generic_name,267977
quantity,215953


,0
code,320749
url,320749
creator,3535
created_t,189567
created_datetime,189568
last_modified_t,180495
last_modified_datetime,180495
product_name,221347
generic_name,38584
quantity,13826


In [ ]:
# @title 1.3 Unique Values Analysis
cdisplay.subtitle(" 1.2 - Unique values Analysis", font_size="22px")
# Display the number of unique values per column
unique_values = data.nunique()

# Calculate the percentage of unique values per column
total_rows = len(data)
unique_percentage = (unique_values / total_rows) * 100

# Display describe for unique values
cdisplay.paragraph("Description of unique values", font_size="18px", color = "Red")
cdisplay.paragraph(display(unique_percentage.describe()))

# Calculate the 50% quantile (median)
median_x = unique_percentage.quantile(0.5)

# Set the colors: Red for columns > 30%, gray for the rest
colors = ['red' if val > 30 else 'gray' for val in unique_percentage]

# Define which indexes to show (e.g., every 5th column)
index_step = 5
shown_indexes = unique_percentage.index[::index_step]

# Create bar plot using go.Bar
fig = Figure(
    data=Bar(
        x=unique_values.index,
        y=unique_percentage,
        marker_color=colors  # Apply the color condition to the bars
    )
)

# Add a horizontal line for the 50% quantile (median)
fig.add_shape(
    type='line',
    x0=unique_percentage.index[0],
    x1=unique_percentage.index[-1],
    y0=median_x,
    y1=median_x,
    line=dict(color='blue', dash='dash'),
    name='50% Quantile (Median)'
)

# Update the layout for the plot to show only a subset of x-axis labels
fig.update_layout(
    title="Percentage of Unique Values per Column (Red means unique values > 30%)",
    xaxis=dict(
        tickvals=shown_indexes,  # Show only every Nth column based on index_step
        ticktext=shown_indexes,  # Display the corresponding labels
        tickangle=-45  # Optional: Rotate the labels for better readability
    ),
    yaxis=dict(title='Percentage of Unique Values (%)'),
)

cdisplay.paragraph(
    "The summary statistics of the unique values show the following characteristics:",
    font_size="18px", color = "Red"
)



# Count
cdisplay.paragraph(
    "1. Count (162): There are 162 observations for the unique values, indicating a relatively large dataset.",
    font_size="14px"
)

# Mean
cdisplay.paragraph(
    "2. Mean (5.25): The average value of the unique values is 5.25, suggesting that on average, "
    "the unique values tend to be on the lower side.",
    font_size="14px"
)

# Standard Deviation
cdisplay.paragraph(
    "3. Standard Deviation (16.55): The high standard deviation indicates a high spread or variability, "
    "with some unique values being much larger than others.",
    font_size="14px"
)

# 75th Percentile
cdisplay.paragraph(
    "4. 75th Percentile (1.12): 75% of the values are below 1.12, further suggesting a concentration "
    "of lower values.",
    font_size="14px"
)

# Maximum
cdisplay.paragraph(
    "5. Maximum (99.99) : The maximum value of 99.99 indicates the presence of some outliers or "
    "extreme values in the dataset.",
    font_size="14px"
)


# Show the plot
fig.show()

,0
count,162.000000
mean,5.252687
std,16.551849
min,0.000000
25%,0.004676
50%,0.096018
75%,1.116837
max,99.992830


In [ ]:
# @title 1.4 Correlation Analysis

import plotly.express as px
import pandas as pd
import numpy as np

cdisplay.subtitle("1.3 - Correlation Analysis:", is_bold=True, font_size="22px", center=True)
cdisplay.paragraph("Correlation HeatMap : ", font_size="18px", color='Red')

# Select numeric columns
numeric_columns = data.select_dtypes(include=['number']).columns

if len(numeric_columns) > 0:
    # Compute the correlation matrix
    correlation_matrix = data[numeric_columns].corr()

    # Create a heatmap with Plotly
    fig = px.imshow(
        correlation_matrix,
        text_auto=True,  # Display the correlation values in each cell
        color_continuous_scale='RdBu',  # Red to blue color scale (diverging for positive and negative)
        zmin=-1, zmax=1,  # Set the correlation limits
        labels={'x': 'Columns', 'y': 'Columns', 'color': 'Correlation'},  # Labels for axes and color
        title="Correlation Matrix of Numerical Features"
    )

    # Adjust the layout for better spacing and presentation
    fig.update_layout(
        title_font_size=20,
        title_x=0.5,  # Center the title
        width=900,  # Decreased width for a smaller plot
        height=900,  # Decreased height for a smaller plot
        xaxis={'tickangle': 45},  # Rotate the x-axis labels
        yaxis={'tickangle': 0},  # Keep the y-axis labels horizontal
        autosize=True,  # Allow resizing
    )

    # Show the plot
    fig.show()

    # Calculate more relevant correlation metrics
    abs_corr_matrix = correlation_matrix.abs()  # Take absolute values for more meaningful analysis

    avg_abs_corr = abs_corr_matrix[~np.eye(
        abs_corr_matrix.shape[0], dtype=bool)].mean().mean()  # Average of absolute correlations excluding the diagonal
    high_corr_pairs = (abs_corr_matrix > 0.8).sum().sum() - len(correlation_matrix)  # Pairs with correlation above 0.8
    weak_corr_pairs = (abs_corr_matrix < 0.2).sum().sum() - len(correlation_matrix)  # Pairs with correlation below 0.2
    zero_corr_pairs = (abs_corr_matrix == 0).sum().sum() - len(correlation_matrix)  # Pairs with correlation exactly 0

    # Display the updated correlation metrics
    cdisplay.paragraph("Correlation Metrics:", is_bold=True, font_size="18px", color='Red')
    cdisplay.paragraph(f"2 - Average Absolute Correlation: {avg_abs_corr:.2f}", font_size="14px")
    cdisplay.paragraph(f"3 - Number of Highly Correlated Pairs (> 0.8): {high_corr_pairs}", font_size="14px")
    cdisplay.paragraph(f"4 - Number of Weakly Correlated Pairs (< 0.2): {weak_corr_pairs}", font_size="14px")
    cdisplay.paragraph(f"5 - Number of Zero Correlations: {zero_corr_pairs}", font_size="14px")

cdisplay.paragraph("Analysis and recommendations : ", font_size="18px", color='Red')

# Display summary
cdisplay.paragraph("1 - Average Absolute Correlation (40%) : Moderate correlation, suggesting some relationships between features, but not overwhelming.", font_size="14px")

cdisplay.paragraph("2 - Highly Correlated Pairs (> 80%) (719 pairs): A large number of highly correlated features, indicating redundancy. We should remove or combine them to simplify the dataset.", font_size="14px")

cdisplay.paragraph("3 - Weakly Correlated Pairs (< 20%) (2224 pairs): A lot of weak correlations, suggesting many independent features. We should drop or transform them.", font_size="14px")

cdisplay.paragraph("4 - Zero Correlations (-104): Unexpected negative value here. which typically indicate irrelevant features. After investigating, we will delete columns (NAN correlation ) that have a big percentage of NaN values.", font_size="14px")

cdisplay.paragraph("After applying all of this, we hope and will verify if we have correlation better metrics, HENCE BETTER DATA ! ", font_size="18px", color="Blue")


In [10]:
# @title 2.1  Feature Selection based on NaN values
cdisplay.title("Section 2 - Data Processing")


# Display the introduction for data preprocessing
cdisplay.paragraph(
    "Section 2 - Data Preprocessing is a crucial stage in the data analysis and "
    "machine learning pipeline, where raw data is cleaned and transformed into a "
    "format suitable for analysis or model building. The primary goal of this section "
    "is to improve data quality by addressing issues such as missing values, irrelevant "
    "features ...",
    font_size="14px",
    is_bold=True
)
# Display the subtitle for SubSection 2.1
cdisplay.subtitle(
    "2.1 - Feature Selection ( Horizontal preprocessing )", font_size="22px"
)
cdisplay.paragraph(
    "In this subsection, we focus on feature selection based on the degree of "
    "missing values (NaN). Since the correlation matrix revealed many features with "
    "zero correlation, we aim to trim the DataFrame by removing columns that contain "
    "a high percentage of missing values. These columns are considered less useful "
    "for analysis and may introduce noise into the model. By removing such features, "
    "we can simplify the dataset and improve model performance, focusing on features "
    "with more relevant and complete information.",
    font_size="14px"
)

# plotting a graph of missing values percentage across all features

# Calculate the median of the percentage of missing values
median_x = missing_percentage.quantile(0.5)

# Set the colors: Red for columns > 30%, gray for the rest
colors = ['red' if val > 30 else 'blue' for val in missing_percentage]

# Define which indexes to show (e.g., every 5th column)
index_step = 5
shown_indexes = missing_percentage.index[::index_step]

# Create bar plot using go.Bar
fig = Figure(
    data=Bar(
        x=missing_percentage.index,
        y=missing_percentage,
        marker_color=colors  # Apply the color condition to the bars
    )
)

# Add a horizontal line for the 50% quantile (median)
fig.add_shape(
    type='line',
    x0=missing_percentage.index[0],
    x1=missing_percentage.index[-1],
    y0=median_x,
    y1=median_x,
    line=dict(color='blue', dash='dash'),
    name='50% Quantile (Median)'
)

# Update the layout for the plot to show only a subset of x-axis labels
fig.update_layout(
    title="Percentage of Missing Values per Column (Red means missing values > 30%)",
    xaxis=dict(
        tickvals=shown_indexes,  # Show only every Nth column based on index_step
        ticktext=shown_indexes,  # Display the corresponding labels
        tickangle=-45  # Optional: Rotate the labels for better readability
    ),
    yaxis=dict(title='Percentage of Missing Values (%)'),
)

cdisplay.paragraph(
    "Feature selection based on missing values percentage:",
    font_size="18px", color="Red"
)
fig.show()
cdisplay.paragraph("We notice that more than 50% have approximatly 99% of missing values, so we will delete every column that has more that 40% of missing values")

# deleting columns with missing values superior to the set threshold, in our case 40%
def delete_irrelevent_columns(data: DataFrame = data, quantile_: float = 0.4) -> DataFrame:
    """
    Deletes columns with missing values above the specified threshold (quantile).

    Parameters:
    data (DataFrame): The input DataFrame to process.
    quantile_ (float): The threshold for missing values as a percentage (default is 40%).

    Returns:
    DataFrame: The DataFrame after removing columns with excessive missing values.
    """
    missing_percentage = data.isnull().sum() / len(data) * 100
    filtered_columns = missing_percentage[missing_percentage < quantile_ * 100].index
    filtered_data = data[filtered_columns]


    # List of columns to delete
    columns_to_delete = [
        'url', 'creator','created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime',
        'product_name', 'brands', 'countries', 'countries_tags', 'brands_tags', 'states',
        'states_tags', 'states_fr', 'ingredients_text', 'serving_size', 'additives_n',
        'additives', 'ingredients_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_n', 'nutrition-score-uk_100g'
    ]

    # Remove the specified columns
    filtered_data = filtered_data.drop(columns=columns_to_delete, errors='ignore')
    cdisplay.paragraph("Custom feature deletion", font_size="18px", color = "Red")
    cdisplay.paragraph(f"we will be deleting {len(columns_to_delete)} columns that would serve for nothing.", font_size="14px")
    cdisplay.paragraph(display(columns_to_delete), font_size="14px")

    return filtered_data

filtered_data = delete_irrelevent_columns(data)
cdisplay.paragraph("Feature selection results :", font_size= "18px", color = "Red")
cdisplay.paragraph("Original number of columns:" +str(len(data.columns)))
cdisplay.paragraph("Columns deleted:"+ str(len(data.columns) -len(filtered_data.columns)))
cdisplay.paragraph("New columns count :"+ str(len(filtered_data.columns)))
cdisplay.paragraph("New Data Frame :", font_size= "18px", color = "Red")
cdisplay.paragraph(display(filtered_data.head()))


['url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'product_name',
 'brands',
 'countries',
 'countries_tags',
 'brands_tags',
 'states',
 'states_tags',
 'states_fr',
 'ingredients_text',
 'serving_size',
 'additives_n',
 'additives',
 'ingredients_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil_n',
 'nutrition-score-uk_100g']

,code,countries_fr,nutrition_grade_fr,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition-score-fr_100g
0,0000000003087,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,États-Unis,d,2243.0,28.57,28.57,64.29,14.29,3.6,3.57,0.00000,0.000,14.0
2,0000000004559,États-Unis,b,1941.0,17.86,0.00,60.71,17.86,7.1,17.86,0.63500,0.250,0.0
3,0000000016087,États-Unis,d,2540.0,57.14,5.36,17.86,3.57,7.1,17.86,1.22428,0.482,12.0
4,0000000016094,États-Unis,NaN,1552.0,1.43,NaN,77.14,NaN,5.7,8.57,NaN,NaN,NaN


In [ ]:
# @title 2.2 Data Cleaning ( Vertical Processing )
cdisplay.subtitle(
    "2.2 -  Data cleaning (Vertical Processing)", font_size="22px"
)
cdisplay.paragraph(
    "In this subsection, we focus on cleaning the dataset vertically, which involves "
    "processing lines rather than rows. Specifically, we will handle missing values (NaN) "
    "and outliers. This process is essential to ensure that the dataset is "
    "consistent and suitable for analysis. We will replace or remove invalid values, handle aberrant "
    "entries, and drop rows with excessive missing data. These steps will result in a cleaner dataset "
    "with more reliable information, improving the overall quality of the data for subsequent modeling and analysis.",
    font_size="14px"
)


# Function for Step 1: Initial number of records
def initial_records(filtered_data):
    init_len = len(filtered_data)
    cdisplay.subtitle("1. Initial Number of Records", color="blue", font_size="18px")
    cdisplay.paragraph(f"Initially, we have {init_len} records.", font_size="14px")
    return filtered_data, init_len

# Function for Step 2: Filter out non-French products
def filter_french_products(filtered_data):
    filtered_data = filtered_data[filtered_data["countries_fr"] == "France"]
    cdisplay.subtitle("2. Filter French Products", color="blue", font_size="18px")
    cdisplay.paragraph(f"After filtering for French products, we have {len(filtered_data)} records.", font_size="14px")
    return filtered_data

# Function for Step 3, 4, and 5: Handle 'energy_100g' aberrant values and invalid values in other columns
def handle_values(filtered_data):
    # Step 3: Maximum 'energy_100g' before handling aberrant values
    cdisplay.subtitle("3. Maximum 'energy_100g' Before Handling Aberrant Values", color="blue", font_size="18px")
    cdisplay.paragraph("Let's first examine the maximum value in the 'energy_100g' column before handling any aberrant values.", font_size="14px")
    max_energy_before = max(filtered_data.dropna()['energy_100g'])
    cdisplay.paragraph(f"The maximum 'energy_100g' value is: {max_energy_before}", font_size="14px")

    # Step 4: Replace aberrant 'energy_100g' values (>3000) with the average value for each nutrition grade
    energy_means = filtered_data.groupby('nutrition_grade_fr')['energy_100g'].transform('mean')
    filtered_data.loc[filtered_data['energy_100g'] > 3000, 'energy_100g'] = energy_means

    # Step 5: Process columns ending with '100g' (except 'energy_100g' and 'nutrition-score-fr_100g')
    columns_100g = [col for col in filtered_data.columns if col.endswith("100g") and col not in ["energy_100g", "nutrition-score-fr_100g"]]
    for col in columns_100g:
        # Identify invalid values (>100 or <0) for each column
        mask = (filtered_data[col] > 100) | (filtered_data[col] < 0)
        medians = filtered_data[~mask].groupby("nutrition_grade_fr")[col].median()
        for grade, median_value in medians.items():
            filtered_data.loc[
                (filtered_data["nutrition_grade_fr"] == grade) & mask, col
            ] = median_value

    # Display updated max value after handling aberrant 'energy_100g'
    cdisplay.subtitle("4. Maximum 'energy_100g' After Handling Aberrant Values", color="blue", font_size="18px")
    cdisplay.paragraph("After replacing values greater than 3000 with the group-wise mean, the maximum 'energy_100g' value is now:", font_size="14px")
    max_energy_after = max(filtered_data.dropna()['energy_100g'])
    cdisplay.paragraph(f"The maximum 'energy_100g' value after correction is: {max_energy_after}", font_size="14px")

    # After processing the '100g' columns for invalid values
    cdisplay.subtitle("5. Handling Invalid Values (>100, <0)", color="blue", font_size="18px")
    cdisplay.paragraph(f"After replacing invalid values (>100 or <0) in other columns, the dataset now contains {len(filtered_data)} records.", font_size="14px")

    return filtered_data

# Function for Step 6: Drop rows where all columns (except 'code' and 'countries_fr') are NaN
def remove_nan_rows(filtered_data):
    columns_to_check = filtered_data.columns.difference(['code', 'countries_fr'])
    filtered_data = filtered_data.dropna(how='all', subset=columns_to_check)
    cdisplay.subtitle("6. Removing Rows with All NaN Values", color="blue", font_size="18px")
    cdisplay.paragraph(f"After removing rows with all NaN values (except for 'code' and 'countries_fr'), the dataset has {len(filtered_data)} records.", font_size="14px")
    return filtered_data

# Function for Step 7: Remove outliers where the sum of '100g' columns exceeds 100g
def remove_outliers(filtered_data):
    ingredient_columns = [
        col for col in filtered_data.columns
        if col.endswith("100g") and col not in ["energy_100g", "nutrition-score-fr_100g"]
    ]
    filtered_data["ingredient_sum"] = filtered_data[ingredient_columns].sum(axis=1)
    outlier_mask = filtered_data["ingredient_sum"] > 100
    num_records_to_delete = outlier_mask.sum()
    filtered_data = filtered_data[~outlier_mask].drop(columns=["ingredient_sum"])

    cdisplay.subtitle("7. Removing Outliers Where Ingredient Sum Exceeds 100g", color="blue", font_size="18px")
    cdisplay.paragraph(f"A total of {num_records_to_delete} records were removed due to outliers.", font_size="14px")
    cdisplay.paragraph("Records where the sum of the '100g' columns (excluding 'energy_100g' and 'nutrition-score-fr_100g') exceeded 100 have been removed.", font_size="14px")
    return filtered_data

# Function for Step 8: Fill NaN values with 0
def fill_nan_values(filtered_data):
    filtered_data = filtered_data.fillna(0)
    max_fat = max(filtered_data['fat_100g'])
    cdisplay.subtitle("8. Filling NaN Values with 0", color="blue", font_size="18px")
    cdisplay.paragraph("Filling all NaN values with 0 and calculating the maximum value of 'fat_100g'.", font_size="14px")
    cdisplay.paragraph(f"The maximum value of 'fat_100g' after filling NaN with 0 is: {max_fat}", font_size="14px")
    return filtered_data

# Function for Step 9: Final comparison of initial and final number of records
def final_comparison(init_len, filtered_data):
    cdisplay.subtitle("9. Final Comparison of Initial and Final Records", color="blue", font_size="18px")
    cdisplay.paragraph(f"Initially, there were {init_len} records.", font_size="14px")
    cdisplay.paragraph(f"After cleaning, we are left with {len(filtered_data)} records.", font_size="14px")
    cdisplay.paragraph(
        "This cleaning process removed invalid records, corrected aberrant values, and filtered out outliers. The final dataset is now more reliable for analysis.",
        font_size="14px"
    )

# Main code to call each function
filtered_data, init_len = initial_records(filtered_data)
filtered_data = filter_french_products(filtered_data)
filtered_data = handle_values(filtered_data)
filtered_data = remove_nan_rows(filtered_data)
filtered_data = remove_outliers(filtered_data)
filtered_data = fill_nan_values(filtered_data)
final_comparison(init_len, filtered_data)


<ipython-input-17-17468ddf523c>:79: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# @title 3.1 Results and Conclusion
# Title of the conclusion

cdisplay.title("Section 3 - Results and Conclusion")
cdisplay.subtitle("3.1 Results :")

# Select numeric columns
numeric_columns = filtered_data.select_dtypes(include=['number']).columns

if len(numeric_columns) > 0:
    # Compute the correlation matrix
    correlation_matrix = filtered_data[numeric_columns].corr()

    # Create a heatmap with Plotly
    fig = px.imshow(
        correlation_matrix,
        text_auto=True,  # Display the correlation values in each cell
        color_continuous_scale='RdBu',  # Red to blue color scale (diverging for positive and negative)
        zmin=-1, zmax=1,  # Set the correlation limits
        labels={'x': 'Columns', 'y': 'Columns', 'color': 'Correlation'},  # Labels for axes and color
        title="Correlation Matrix of Numerical Features"
    )

    # Adjust the layout for a smaller figure
    fig.update_layout(
        title_font_size=20,
        title_x=0.5,  # Center the title
        width=800,  # Decreased width for a smaller plot
        height=800,  # Decreased height for a smaller plot
        xaxis={'tickangle': 45},  # Rotate the x-axis labels
        yaxis={'tickangle': 0},  # Keep the y-axis labels horizontal
        autosize=True,  # Allow resizing
    )

    # Show the plot
    fig.show()

# Display the improved correlation metrics
cdisplay.paragraph("After preprocessing, we observed significant improvements in the dataset's structure:", font_size="22px")

# Display the comparison of old vs new metrics
cdisplay.paragraph("Before Preprocessing:", font_size="20px")
cdisplay.paragraph(f"1 - Average Absolute Correlation: 0.40", font_size="14px")
cdisplay.paragraph(f"2 - Highly Correlated Pairs (> 0.8): 719", font_size="14px")
cdisplay.paragraph(f"3 - Weakly Correlated Pairs (< 0.2): 2224", font_size="14px")
cdisplay.paragraph(f"4 - Zero Correlations: -104", font_size="14px")

cdisplay.paragraph("After Preprocessing:", font_size="20px")
cdisplay.paragraph(f"1 - Average Absolute Correlation: 0.28", font_size="14px")
cdisplay.paragraph(f"2 - Highly Correlated Pairs (> 0.8): 2", font_size="14px")
cdisplay.paragraph(f"3 - Weakly Correlated Pairs (< 0.2): 46", font_size="14px")
cdisplay.paragraph(f"4 - Zero Correlations: -10", font_size="14px")
cdisplay.subtitle("3.2 - Discussions & Conclusion")
cdisplay.paragraph("""
The preprocessing steps have notably transformed the dataset by eliminating irrelevant features and reducing redundancy in the relationships between variables.
""", font_size="14px")
cdisplay.paragraph("Before the preprocessing, the data showed a high degree of correlation, with 719 highly correlated pairs and 2224 weakly correlated ones, indicating substantial overlap among the features. \n \n")
cdisplay.paragraph('After cleaning the data, we achieved a much leaner and more informative dataset, with only 2 highly correlated pairs and a reduced number of weak correlations. This cleanup reduced the dimensionality, making the dataset more interpretable and ready for further analysis. ')
cdisplay.paragraph("Furthermore, by removing zero correlations and addressing missing or invalid values, we improved the overall quality of the dataset. ")
cdisplay.paragraph('In conclusion, this process has successfully simplified the dataset while retaining the most important information for analysis, and these improvements demonstrate the crucial role of data cleaning and feature selection in any data-driven project.')